## PS3

In problem set 1, we illustrated how to perform integration using Monte Carlo Methods. We also showed how to smooth out the function involving sharp peaks by utilizing a predefined weight function to decrease the standard deviation.
In this notebook, we are trying to minimize the variance of the integration further by dividing the integration domain into 10 equal pieces and perform integration for each interval seperately.

To illustrate the idea, we used the following integral

$$\int_{2}^{10}(x-5)\cdot e^{-(\frac{x}{2}-3)}dx$$

For each interval we used linear weight functions ${w_1,w_2,..,w_{10}}$ where $w_i$ is the weight function for i'th interval. We also have the following normalization,

$$\int_{x_{i-1}}^{x_i} w_i(x)dx=x_i-x_{i-1}$$ and following boundary conditions,

$$w(x_{i-1})=f(x_{i-1})\text{ }\text{ and }\text{ } w(x_{i})=f(x_{i}) $$


Then the final integral can be calculated as

$$I_{Total}=\sum_{i=1}^{10} I_i$$
where 
$$I_i=\int_{x_{i-1}}^{x_i} f(x)dx$$

and this integral is to be calculated using our standard MC integration method using $w_i(x)$ as the weight function.




Following code snippet illustrates the described method.

In [8]:
import numpy as np
from numpy.polynomial import Polynomial as P
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff


#Integrand function
def f(x):
    return (x-5)*np.exp(-(x/2-3))+10
    
#Calculates the coefficients of linear weight function.    
def findw(f,lower,upper):
    #Find the linear function.
    slope=(f(upper)-f(lower))/upper-lower
    #Normalization.
    A=(slope/2)*(upper**2)-(slope*upper-f(upper))*upper-((slope/2)*(lower**2)-(slope*upper-f(upper))*lower)
    A/=(upper-lower)
    return [slope/A, (-slope*upper+f(upper))/A] 

#Performs integration.
def integrate(f,w,lower,upper,N):
    #Generate uniform random inputs.
    inputs=(upper-lower)*np.random.rand(N)+lower
    #Calculate integration constant using boundary conditions.
    C=lower-((w[0]/2)*lower**2+w[1]*lower)
    #Find the inverse X.
    p= [P([-i+C,w[1],w[0]/2]) for i in inputs]
    inverse_inputs=[]
    for i in p:
        if i.roots()[0]>=lower and i.roots()[0]<=upper:
            inverse_inputs.append(i.roots()[0])
        else: 
            inverse_inputs.append(i.roots()[1])
    inverse_inputs=np.array(inverse_inputs)
    #Calculate f(inverse(x))/w(inverse(x)).
    outputsF=f(inverse_inputs)
    outputsW=w[0]*(inverse_inputs)+w[1]
    outputs=outputsF/outputsW
    #Store generated points for variance calculation.
    Vsum=outputs.sum()
    return (upper-lower)*Vsum/N , outputsF-10/outputsW

#Divide the region into 10 pieces.
l=np.arange(2,10.01,0.8)   
#Real value of the integral
I_real=-16.673
#N values
N=[10,100,500,1000,10000]
#Integration results.
results=[]
#All generated points.
points=[]
#Standart deviation values
sigmas=[]
for k in N:
    I=0
    for i in range (0,len(l)-1):
        w=findw(f,l[i],l[i+1])
        temp,temp2=integrate(f,w,l[i],l[i+1],k)
        I+=temp
        points=np.concatenate((points,temp2))
    var=np.var(points)/k
    I=I-80
    results.append(I)
    sigmas.append(np.sqrt(var))
    print(10*k,I,I-I_real,np.sqrt(var),(I-I_real)/np.sqrt(var))
  


100 -17.9368203546 -1.26382035458 1.80025702265 -0.702022177213
1000 -16.6083170563 0.0646829437471 0.558323998322 0.115851985481
5000 -16.6435324663 0.0294675337124 0.252617729847 0.116648715552
10000 -16.4562308065 0.216769193529 0.17899511069 1.21103416006
100000 -16.7051932257 -0.0321932256809 0.0566317592838 -0.568465929508


In [9]:
data_matrix = [["N", "I MC", "I Real", "I MC-I Real","sigma","I MC-I Real/sigma"],
            ["100", results[0], I_real,results[0]-I_real,sigmas[0],(I-I_real)/sigmas[0]],
            ["1000", results[1], I_real,results[1]-I_real,sigmas[1],(I-I_real)/sigmas[1]],
               ["5000", results[2], I_real,results[2]-I_real,sigmas[2],(I-I_real)/sigmas[2]],
               ["10000", results[3], I_real,results[3]-I_real,sigmas[3],(I-I_real)/sigmas[3]],
               ["100000", results[4], I_real,results[4]-I_real,sigmas[4],(I-I_real)/sigmas[4]]]
plotly.tools.set_credentials_file(username='guneykan', api_key='Yu3MsgD6Zlfbb0B3S5Mx')
table = ff.create_table(data_matrix)
py.iplot(table)